In [2]:
from lambda_function import *

In [28]:
params = {'dynamo_table_name': 'temp-capture-senado-proposicoes-lista-historical-2021-02-11-17-01-47', 'order': 0}

In [29]:
print(params)

# Para poder identificar os erros que acontecerão no dynamo:
dynamo_exceptions = boto3.client('dynamodb').exceptions

try:
    # Input `params` default (temp table):
    if 'dynamo_table_name' in params.keys():           
        if debug:
            print('Loading params from dynamo temp table...')
        # Carrega dicionário do dynamo:
        event = load_params(params)
    # For debugging:
    else:
        if debug:
            print('Assuming `params` is a typical data in dynamo temp table item.')
        # Se não existe referência à tabela no dynamo, assume que esse é o próprio dicionário
        # (opção para debugging):
        event = params
        params = {'order': 0}

# Possível erro: não encontrou a tabela temp no DynamoDB:
except dynamo_exceptions.ResourceNotFoundException:

    print('DynamoDB Table does not exist')    

# Algum outro possível erro:
except Exception as e:

    # Raise error somewhere, maybe slack
    print(e)


{'dynamo_table_name': 'temp-capture-senado-proposicoes-lista-historical-2021-02-11-17-01-47', 'order': 0}
Loading params from dynamo temp table...


In [30]:
# Set max_retries for HTTP GET to 3 (instead of one):
# This makes the download more robust.
session = requests.Session()
session.mount('http', requests.adapters.HTTPAdapter(max_retries=3))

# Pega o arquivo especificado pelo url no event:
if 'url' in event.keys():
    if debug:
        print('GET file...')
    try:
        response = session.get(event['url'], 
                               params=event['params'], 
                               headers=event['headers'], # configs para HTTP GET.
                               timeout=600)
    except requests.exceptions.SSLError:
        response = session.get(event['url'], 
                               params=event['params'], 
                               headers=event['headers'], # configs para HTTP GET.
                               timeout=600,
                               verify=False)


GET file...


In [31]:
# Se captura ocorreu bem ou se ainda vai capturar (no caso sem url),
# salva na AWS S3 e Google Storage:
if response == None or (response != None and response.status_code == 200):
    if debug:
        if response != None:
            print('GET successful. Writing to S3...')
        else:
            print('Will obtain non-http-get data...')
    # Salva arquivo baixado no S3 (Amazon), além de outras coisas:
    # (também registra o destino do arquivo)
    status_code_s3 = write_to_s3(event, response)
    if debug:
        print('write_to_s3 status code:', status_code_s3)

    # Copy the result to GCP storage:
    status_code_gcp = 10
    if status_code_s3 == 200:
        status_code_gcp = copy_s3_to_storage_gcp(params['order'], event['bucket'], event['key'])

    # Registra url capturado em tabela do dynamo, se tal ação for requisitada.
    # Isso acontece no caso da captura de matérias do DOU. O motivo para 
    # guardarmos quais matérias foram baixadas é que as matérias podem ser 
    # publicadas em horários diferentes e o site do DOU pode sair do ar.
    # Para não perder nenhuma matéria, vamos registrando quais do dia de hoje 
    # já baixamos:
    if 'url_list' in event['aux_data'].keys():
        if status_code_gcp == 200:
            if debug:
                print('Register sucessful capture on table' + event['aux_data']['url_list'])
            register_captured_url_aws(event['aux_data']['url_list'], event['url'])
        elif debug:
            print('Capture failed for ' + event['url'])


GET successful. Writing to S3...
brutos-publicos legislativo/senado/v1/proposicoes-lista/senado-proposicoes-lista_2019.json
Creating json list...
Putting object in S3 bucket...
s3_log: {'ResponseMetadata': {'RequestId': 'CGDY6X6N7KDG5H5R', 'HostId': 'gGwnwmociKMltZczGpU7G43KRkvNhPES6csooH6S9TFhAl4f+3D4OssGbCgvp2GkbhRlrZJ4U1g=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'gGwnwmociKMltZczGpU7G43KRkvNhPES6csooH6S9TFhAl4f+3D4OssGbCgvp2GkbhRlrZJ4U1g=', 'x-amz-request-id': 'CGDY6X6N7KDG5H5R', 'date': 'Thu, 11 Feb 2021 20:21:05 GMT', 'etag': '"7285e02e87c1d998fe8b26a38272aa6c"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"7285e02e87c1d998fe8b26a38272aa6c"'}
write_to_s3 status code: 200
Invoking write-to-storage-gcp...
